<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#1.-100-HOT-songs:-Create-DB" data-toc-modified-id="1.-100-HOT-songs:-Create-DB-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>1. 100 HOT songs: Create DB</a></span><ul class="toc-item"><li><span><a href="#1.1.-Scrapping-web-Hot-100-Songs-to-obtain-the-Hot-Song-initial-DB" data-toc-modified-id="1.1.-Scrapping-web-Hot-100-Songs-to-obtain-the-Hot-Song-initial-DB-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>1.1. Scrapping web Hot 100 Songs to obtain the Hot Song initial DB</a></span></li><li><span><a href="#1.2.-100-HOT-songs:-implement-audio-features--API-Spotify" data-toc-modified-id="1.2.-100-HOT-songs:-implement-audio-features--API-Spotify-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>1.2. 100 HOT songs: implement audio features- API Spotify</a></span></li><li><span><a href="#1.3.-Get-ID's-from-Hot-Songs" data-toc-modified-id="1.3.-Get-ID's-from-Hot-Songs-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>1.3. Get ID's from Hot Songs</a></span></li><li><span><a href="#1.3.2.-Get-audio-feaures-for-all-ID's-for-our-Hot-Songs" data-toc-modified-id="1.3.2.-Get-audio-feaures-for-all-ID's-for-our-Hot-Songs-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>1.3.2. Get audio feaures for all ID's for our Hot Songs</a></span></li><li><span><a href="#2.4.-Merging-audio-features-with-previous-DF-for-Hot-Songs" data-toc-modified-id="2.4.-Merging-audio-features-with-previous-DF-for-Hot-Songs-1.5"><span class="toc-item-num">1.5&nbsp;&nbsp;</span>2.4. Merging audio features with previous DF for Hot Songs</a></span></li></ul></li><li><span><a href="#3.-Spotify:-Create-DB" data-toc-modified-id="3.-Spotify:-Create-DB-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>3. Spotify: Create DB</a></span><ul class="toc-item"><li><span><a href="#3.1.-Retrive-tracks-from-playlist" data-toc-modified-id="3.1.-Retrive-tracks-from-playlist-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>3.1. Retrive tracks from playlist</a></span></li><li><span><a href="#3.2.-Get-audio-features-for-playlist" data-toc-modified-id="3.2.-Get-audio-features-for-playlist-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>3.2. Get audio features for playlist</a></span></li><li><span><a href="#3.3.-Put-all-together" data-toc-modified-id="3.3.-Put-all-together-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>3.3. Put all together</a></span></li></ul></li><li><span><a href="#3.-PUT-BOTH-DB-TOGETHER-FOR-K-MEANS-AND-GET-LABELS" data-toc-modified-id="3.-PUT-BOTH-DB-TOGETHER-FOR-K-MEANS-AND-GET-LABELS-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>3. PUT BOTH DB TOGETHER FOR K-MEANS AND GET LABELS</a></span><ul class="toc-item"><li><span><a href="#3.1.-Concatenate-data-frames" data-toc-modified-id="3.1.-Concatenate-data-frames-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>3.1. Concatenate data frames</a></span></li><li><span><a href="#5.2-Select-the-audio-features-interested-(train-model-in-another-file)" data-toc-modified-id="5.2-Select-the-audio-features-interested-(train-model-in-another-file)-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>5.2 Select the audio features interested (train model in another file)</a></span></li><li><span><a href="#4.4-Add-labels-and-export-each-DB" data-toc-modified-id="4.4-Add-labels-and-export-each-DB-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>4.4 Add labels and export each DB</a></span></li></ul></li></ul></div>

# Project Week 3 - Desired music recommender

## 1. 100 HOT songs: Create DB

We are gonna create a function to scrape the [Billaboards 100 HOT songs](https://www.billboard.com/charts/hot-100) and create a local dataframe of songs with them inluding:

- Song's name
- Song's artist

After we will parse Spotify's API to get audio features for this songs and their Id.

In [145]:
# 1. import libraries
import requests # to download html code
from bs4 import BeautifulSoup # to navigate through the html code
import pandas as pd
import numpy as np
import re

In [146]:
# 2. find url and store it in a variable
url = "https://www.billboard.com/charts/hot-100"

In [147]:
# 3. download html with a get request. Use the function request.get() and store the output in response
response = requests.get(url)
# 200 status code means OK! response.status_code
print(response.status_code)

200


In [148]:
# 4.1. parse html (create the 'soup')
soup = BeautifulSoup(response.text, 'html.parser')
# 4.2. check that the html code looks like it should
#print(soup.prettify())#it is too much, so I will skip this

### 1.1. Scrapping web Hot 100 Songs to obtain the Hot Song initial DB

In [143]:
def scrapp_hotsongs():
    url = "https://www.billboard.com/charts/hot-100"
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    titles_songs = soup.find_all("span", {"class": "chart-element__information__song text--truncate color--primary"})
    songs = [elem.get_text() for elem in titles_songs]
    artists_names = soup.find_all("span", {"class": "chart-element__information__artist"})
    artists = [elem.get_text() for elem in artists_names]
    return songs, artists

def obtain_hotsong_db(list_of_list):
    songs,artists = list_of_list
    dic_songs = {"title": songs, "artist": artists}
    hot_songs = pd.DataFrame(dic_songs)
    return hot_songs


In [144]:
hot_songs = obtain_hotsong_db(scrapp_hotsongs())

### 1.2. 100 HOT songs: implement audio features- API Spotify

Before that we will need to get credentials for our account. Create a python file with the credentials store in the same folder, but remember to include this file on your gitignore to don't share it after on your repo. 

In [17]:
#Access to my file with the Credentials
import sys
sys.path.insert(1, "/Users/lauratll/Documents/Data_Analysis/IronHack/Week_3/Project_Week_3/Credentials/")
from config import *

In [18]:
#Import the requested libraries to work
#The installer: conda install -c conda-forge spotipy

import spotipy
import json
from spotipy.oauth2 import SpotifyClientCredentials
import pprint

In [19]:
#Initialize SpotiPy with user credentias
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id= Client_ID,
                                                           client_secret= Client_Secret))

### 1.3. Get ID's from Hot Songs

In [149]:
#We need to use our lists of Hot Songs, We will get arttis name, id and name of the song from the Tiles

list_hot_songs = hot_songs["title"].to_list()
# list_hot_artists = hot_songs["Artist"].to_list()

#track_info = sp.search(q=list_hot_songs)
#pprint.pprint(track_info)

In [33]:
def get_id_name_title_from_track(tracks_list):
    '''
    With this function we get the ids, names and artist of all out songs in the initial Db from the web scrapping.
    We will input a list of the titles of the songs.
    We will get the artist to make sure we have all the information propertly to be able to after pass those
    to the function that will get the audio features.
    Input: list of all the name of the songs
    Ouput: a list for id, another for names, another for artist
    '''
    list_id = [] 
    list_names = []
    list_artists = []
    for track in tracks_list:
        track_info = sp.search(q=track,limit=1)
#         seconds_to_wait = randint(2,7)
#         sleep(seconds_to_wait)
        track_id = track_info["tracks"]["items"][0]["id"]
        track_name = track_info["tracks"]["items"][0]["name"]
        track_artist = track_info["tracks"]["items"][0]["artists"]
        track_artist2 =[elem["name"] for elem in track_artist]
        track_artist3 = ", ".join(track_artist2)
        list_id.append(track_id)
        list_names.append(track_name)
        list_artists.append(track_artist3)
    return list_id,list_names,list_artists

In [150]:
id_hot,title_hot,artist_hot = get_id_name_title_from_track(list_hot_songs)

### 1.3.2. Get audio feaures for all ID's for our Hot Songs

In [152]:
audio_features_info= sp.audio_features(tracks=id_hot)
# audio_features_info

In [153]:
hot_extra_features = pd.DataFrame(audio_features_info)
hot_extra_features["SongId"] = id_hot
hot_extra_features["title"] = title_hot
hot_extra_features["artists"] = artist_hot

In [154]:
hot_extra_features.head()

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,...,type,id,uri,track_href,analysis_url,duration_ms,time_signature,SongId,title,artists
0,0.759,0.459,8,-5.187,1,0.0948,0.00323,0.0,0.0906,0.695,...,audio_features,2bgTY4UwhfBYhGT4HUYStN,spotify:track:2bgTY4UwhfBYhGT4HUYStN,https://api.spotify.com/v1/tracks/2bgTY4UwhfBY...,https://api.spotify.com/v1/audio-analysis/2bgT...,164442,4,2bgTY4UwhfBYhGT4HUYStN,Butter,BTS
1,0.556,0.661,6,-5.052,0,0.2040,0.30000,0.0,0.1010,0.668,...,audio_features,6PERP62TejQjgHu81OHxgM,spotify:track:6PERP62TejQjgHu81OHxgM,https://api.spotify.com/v1/tracks/6PERP62TejQj...,https://api.spotify.com/v1/audio-analysis/6PER...,178148,4,6PERP62TejQjgHu81OHxgM,good 4 u,Olivia Rodrigo
2,0.702,0.825,6,-3.787,0,0.0601,0.00883,0.0,0.0674,0.915,...,audio_features,463CkQjx2Zk1yXoBuierM9,spotify:track:463CkQjx2Zk1yXoBuierM9,https://api.spotify.com/v1/tracks/463CkQjx2Zk1...,https://api.spotify.com/v1/audio-analysis/463C...,203064,4,463CkQjx2Zk1yXoBuierM9,Levitating (feat. DaBaby),"Dua Lipa, DaBaby"
3,0.677,0.696,0,-6.181,1,0.1190,0.32100,0.0,0.4200,0.464,...,audio_features,4iJyoBOLtHqaGxP12qzhQI,spotify:track:4iJyoBOLtHqaGxP12qzhQI,https://api.spotify.com/v1/tracks/4iJyoBOLtHqa...,https://api.spotify.com/v1/audio-analysis/4iJy...,198082,4,4iJyoBOLtHqaGxP12qzhQI,Peaches (feat. Daniel Caesar & Giveon),"Justin Bieber, Daniel Caesar, Giveon"
4,0.586,0.616,5,-7.964,1,0.0324,0.18200,0.0,0.0927,0.719,...,audio_features,7MAibcTli4IisCtbHKrGMh,spotify:track:7MAibcTli4IisCtbHKrGMh,https://api.spotify.com/v1/tracks/7MAibcTli4Ii...,https://api.spotify.com/v1/audio-analysis/7MAi...,242096,4,7MAibcTli4IisCtbHKrGMh,Leave The Door Open,"Bruno Mars, Anderson .Paak, Silk Sonic"


In [155]:
hot_extra_features.columns

Index(['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness',
       'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo',
       'type', 'id', 'uri', 'track_href', 'analysis_url', 'duration_ms',
       'time_signature', 'SongId', 'title', 'artists'],
      dtype='object')

### 2.4. Merging audio features with previous DF for Hot Songs

In [348]:
hot_songs_df= hot_songs.join(hot_extra_features.iloc[:,:-2]) 
# In the hot_extra_features data frame, the info of th last columns is repeated 
#(title and artist), so I will just sliced the
hot_songs_df.head(10)

,title,artist,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,...,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature,SongId
0,Butter,BTS,0.759,0.459,8,-5.187,1,0.0948,0.00323,0.000000,...,0.695,109.997,audio_features,2bgTY4UwhfBYhGT4HUYStN,spotify:track:2bgTY4UwhfBYhGT4HUYStN,https://api.spotify.com/v1/tracks/2bgTY4UwhfBY...,https://api.spotify.com/v1/audio-analysis/2bgT...,164442,4,2bgTY4UwhfBYhGT4HUYStN
1,Good 4 U,Olivia Rodrigo,0.556,0.661,6,-5.052,0,0.2040,0.30000,0.000000,...,0.668,168.560,audio_features,6PERP62TejQjgHu81OHxgM,spotify:track:6PERP62TejQjgHu81OHxgM,https://api.spotify.com/v1/tracks/6PERP62TejQj...,https://api.spotify.com/v1/audio-analysis/6PER...,178148,4,6PERP62TejQjgHu81OHxgM
2,Levitating,Dua Lipa Featuring DaBaby,0.702,0.825,6,-3.787,0,0.0601,0.00883,0.000000,...,0.915,102.977,audio_features,463CkQjx2Zk1yXoBuierM9,spotify:track:463CkQjx2Zk1yXoBuierM9,https://api.spotify.com/v1/tracks/463CkQjx2Zk1...,https://api.spotify.com/v1/audio-analysis/463C...,203064,4,463CkQjx2Zk1yXoBuierM9
3,Peaches,Justin Bieber Featuring Daniel Caesar & Giveon,0.677,0.696,0,-6.181,1,0.1190,0.32100,0.000000,...,0.464,90.030,audio_features,4iJyoBOLtHqaGxP12qzhQI,spotify:track:4iJyoBOLtHqaGxP12qzhQI,https://api.spotify.com/v1/tracks/4iJyoBOLtHqa...,https://api.spotify.com/v1/audio-analysis/4iJy...,198082,4,4iJyoBOLtHqaGxP12qzhQI
4,Leave The Door Open,Silk Sonic (Bruno Mars & Anderson .Paak),0.586,0.616,5,-7.964,1,0.0324,0.18200,0.000000,...,0.719,148.088,audio_features,7MAibcTli4IisCtbHKrGMh,spotify:track:7MAibcTli4IisCtbHKrGMh,https://api.spotify.com/v1/tracks/7MAibcTli4Ii...,https://api.spotify.com/v1/audio-analysis/7MAi...,242096,4,7MAibcTli4IisCtbHKrGMh
5,Save Your Tears,The Weeknd & Ariana Grande,0.650,0.825,0,-4.645,1,0.0325,0.02150,0.000024,...,0.593,118.091,audio_features,37BZB0z9T8Xu7U3e65qxFy,spotify:track:37BZB0z9T8Xu7U3e65qxFy,https://api.spotify.com/v1/tracks/37BZB0z9T8Xu...,https://api.spotify.com/v1/audio-analysis/37BZ...,191014,4,37BZB0z9T8Xu7U3e65qxFy
6,Kiss Me More,Doja Cat Featuring SZA,0.762,0.701,8,-3.541,1,0.0286,0.23500,0.000158,...,0.742,110.968,audio_features,748mdHapucXQri7IAO8yFK,spotify:track:748mdHapucXQri7IAO8yFK,https://api.spotify.com/v1/tracks/748mdHapucXQ...,https://api.spotify.com/v1/audio-analysis/748m...,208867,4,748mdHapucXQri7IAO8yFK
7,Astronaut In The Ocean,Masked Wolf,0.778,0.695,4,-6.865,0,0.0913,0.17500,0.000000,...,0.472,149.996,audio_features,3Ofmpyhv5UAQ70mENzB277,spotify:track:3Ofmpyhv5UAQ70mENzB277,https://api.spotify.com/v1/tracks/3Ofmpyhv5UAQ...,https://api.spotify.com/v1/audio-analysis/3Ofm...,132780,4,3Ofmpyhv5UAQ70mENzB277
8,Deja Vu,Olivia Rodrigo,0.442,0.612,2,-7.222,1,0.1120,0.58400,0.000006,...,0.178,180.917,audio_features,6HU7h9RYOaPRFeh0R3UeAr,spotify:track:6HU7h9RYOaPRFeh0R3UeAr,https://api.spotify.com/v1/tracks/6HU7h9RYOaPR...,https://api.spotify.com/v1/audio-analysis/6HU7...,215507,4,6HU7h9RYOaPRFeh0R3UeAr
9,Yonaguni,Bad Bunny,0.644,0.648,1,-4.601,1,0.1180,0.27600,0.000000,...,0.440,179.951,audio_features,2JPLbjOn0wPCngEot2STUS,spotify:track:2JPLbjOn0wPCngEot2STUS,https://api.spotify.com/v1/tracks/2JPLbjOn0wPC...,https://api.spotify.com/v1/audio-analysis/2JPL...,206710,4,2JPLbjOn0wPCngEot2STUS


In [349]:
hot_songs_df.columns

Index(['title', 'artist', 'danceability', 'energy', 'key', 'loudness', 'mode',
       'speechiness', 'acousticness', 'instrumentalness', 'liveness',
       'valence', 'tempo', 'type', 'id', 'uri', 'track_href', 'analysis_url',
       'duration_ms', 'time_signature', 'SongId'],
      dtype='object')

In [350]:
#drop extra and repeated columns
hot_songs_df=hot_songs_df.drop(columns=["type","time_signature","analysis_url","track_href","SongId"])

In [351]:
hot_songs_df.tail(10)

,title,artist,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,id,uri,duration_ms
90,Chasing After You,Ryan Hurd With Maren Morris,0.596,0.679,4,-6.395,1,0.0394,0.72300,0.000000,0.1160,0.4790,131.912,1ZIKpMTRPqhwAoVXWY3tXI,spotify:track:1ZIKpMTRPqhwAoVXWY3tXI,207974
91,Tell Em,Cochise & $NOT,0.672,0.717,5,-7.476,1,0.2260,0.10300,0.000000,0.3980,0.4730,157.905,7nc7mlSdWYeFom84zZ8Wr8,spotify:track:7nc7mlSdWYeFom84zZ8Wr8,180380
92,4 Da Gang,42 Dugg & Roddy Ricch,0.871,0.545,7,-10.489,1,0.1890,0.01430,0.000000,0.1480,0.7220,100.482,4KIkADzoVm3A10E9qyjsQ5,spotify:track:4KIkADzoVm3A10E9qyjsQ5,155720
93,Tombstone,Rod Wave,0.550,0.637,8,-5.212,1,0.1630,0.59700,0.000000,0.1290,0.5350,84.448,5pRTz4O2qOkr5nawemy6Wb,spotify:track:5pRTz4O2qOkr5nawemy6Wb,160078
94,What's Next,Drake,0.777,0.580,0,-6.928,0,0.0525,0.01250,0.000000,0.1610,0.0636,129.918,3mDFLytDotXo2p0rvfGbkA,spotify:track:3mDFLytDotXo2p0rvfGbkA,178154
95,Things A Man Oughta Know,Lainey Wilson,0.659,0.683,3,-5.623,1,0.0312,0.51300,0.000005,0.1330,0.3970,139.931,5QS8PNEWbqTEZyQ6e9ZbJf,spotify:track:5QS8PNEWbqTEZyQ6e9ZbJf,203373
96,Country Again,Thomas Rhett,0.467,0.481,2,-6.947,1,0.0255,0.22700,0.000189,0.1100,0.2480,147.886,3CiF9D8iuVhZWDvvjf8HpG,spotify:track:3CiF9D8iuVhZWDvvjf8HpG,221240
97,Drunk (And I Don't Wanna Go Home),Elle King & Miranda Lambert,0.612,0.884,5,-4.400,1,0.0459,0.00516,0.000000,0.0997,0.6260,119.991,0QULNNd9z5s35entfiiXoa,spotify:track:0QULNNd9z5s35entfiiXoa,245626
98,If You Want To,Lil Baby & Lil Durk,0.795,0.599,6,-7.819,1,0.1230,0.04140,0.000000,0.1930,0.3510,97.033,1EXRmdtTA2IEnYGiuw9plD,spotify:track:1EXRmdtTA2IEnYGiuw9plD,195454
99,Seeing Green,"Nicki Minaj, Drake & Lil Wayne",0.573,0.867,5,-2.876,0,0.1680,0.08530,0.000000,0.3800,0.2310,74.861,1TZv3bujNaYz646eezRE91,spotify:track:1TZv3bujNaYz646eezRE91,339506


## 3. Spotify: Create DB

We want to create a df with arouns 10.000 song with following info:
- title
- artist
- all the audio features

### 3.1. Retrive tracks from playlist

In [159]:
from time import sleep
from random import randint
import re

In [160]:
def get_ids_playlists_from_txt(path_file):
    '''
    This function will open a txt file with the link of all the playslits, retrive the playlist ID.
    Output: list with all the playlists ID's
    '''
    f = open(path_file)
    playlists = f.read()
    playlists = playlists.split("\n")
    playlists = [re.findall(r"\/(\d.*)\?",playlist) for playlist in playlists]
    playlists = [elem[0] for elem in playlists]
    return playlists

In [163]:
playlists = get_ids_playlists_from_txt("../data/playlists.txt")

In [162]:
len(playlists)

44

In [50]:
def get_playlist_tracks(username, playlist_id):
    '''
    This function will allow us to get the tracks from one playlist and navigate thought all the pages, as
    otherwise, it will be just able to retrieve information from one of the pages (every pages has a limit of songs)
    Output = All the information from the tracks in a playlist
    '''
    results = sp.user_playlist_tracks(username,playlist_id)
    tracks = results['items'] #list, and we will extend this list
    while results['next']: #while I have a next
        results = sp.next(results)
        seconds_to_wait = randint(2,7)
        sleep(seconds_to_wait)
        tracks.extend(results['items']) #extend my list to the results before. items
    return tracks



In [51]:
def parse_all_info_playlists(playlist_list):
    '''
    This function will allow us to make a big list with all the information from all the tracks from all playlists.
    Input: list of playlist ID's
    Output: A list with all tracks information
    '''
    all_results=[]
    for playlist in playlist_list:
        results =get_playlist_tracks("spotify",playlist)
        all_results.extend(results)
        
    return all_results

In [164]:
all_info_tracks = parse_all_info_playlists(playlists)


In [165]:
def get_info_from_the_tracks_from_parsed_info_playlist(tracks):
    '''
    Once we have parsed all information  we can use this function to get a list of id's, artist name and
    song name of each track in the playlist.
    '''
    name_list= []
    id_list = []
    artist_list = []

    for item in tracks:#results["items"]: #iterate over this, each song entry
        track_name = item["track"]["name"] #nombre de la cancion
        track_id = item["track"]["id"] #id de la cancion
        track_artist = item["track"]["artists"][0]["name"]
        name_list.append(track_name)
        id_list.append(track_id)
        artist_list.append(track_artist)
    return name_list,id_list,artist_list
    

In [166]:
title_spoty,id_spoty,artist_spoty = get_info_from_the_tracks_from_parsed_info_playlist(all_info_tracks)

In [167]:
tracks_spoty=pd.DataFrame({"title":title_spoty,"Id":id_spoty,"artist":artist_spoty})
tracks_spoty.tail()

,title,Id,artist
10666,Hermit the Frog,4Zcz6saEkOII3PlXd9gN3o,MARINA
10667,deja vu,61KpQadow081I2AsbeLcsb,Olivia Rodrigo
10668,WHOLE LOTTA MONEY,5yorXJWdBan1Vlh116ZtQ7,BIA
10669,Slumber Party (feat. Princess Nokia),11ZulcYY4lowvcQm4oe3VJ,Ashnikko
10670,telepatía,6tDDoYIxWvMLTdKpjFkc1B,Kali Uchis


In [169]:
len(tracks_spoty)
# tracks_spoty.iloc[1960:1980,:]

10671

In [170]:
tracks_spoty.isna().sum()

title     0
Id        2
artist    0
dtype: int64

### 3.2. Get audio features for playlist

We can just get 100 in every query, so we are gonna have to itereate every 100 ids and give a new 100 ids again.

In [171]:
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]


In [172]:
#i get my chunks of 100 songs

id_spotify_chunk_100 = list(chunks(id_spoty, 100))
len(id_spotify_chunk_100)

107

In [174]:
#I need to change those two values as the id was not retrieve, it gives None, and then gives 
#an error when we try to get the audio features
id_spotify_chunk_100[19][68] ='0yfNXxlyXdmP0ue1iJijx1' 
id_spotify_chunk_100[19][67] ='0yfNXxlyXdmP0ue1iJijx1'
#Afterward I will delete this songs form the data set, but I need to keep them to mantain the order for the
#next step that is to get the audio features

In [ ]:
if not play_dict_skaterstats:
    play_dict_skaterstats = {}
    play_dict_skaterstats['timeOnIce'] = None
    play_dict_skaterstats['assists'] = None

In [176]:
def get_audio_features_from_id(id_chunks_of_100):
    '''
    This function takes chunks of 100 id's from the tracks and retruns the audio features in 
    '''
    first=True
    for chunk in id_chunks_of_100:
        audio_features_spoty= sp.audio_features(tracks=chunk)
        if first:
            all_features = audio_features_spoty
            first = False
        else:
            all_features.extend(audio_features_spoty)
    return all_features

In [177]:
all_features = get_audio_features_from_id(id_spotify_chunk_100)

In [248]:
len(all_features)

10671

In [271]:
#I get an error :AttributeError: 'NoneType' object has no attribute 'keys') when I try to create the dataframe.

#As I am not able to fix the problem without losing the order...
#I found the elements that are non type, I will try to change them with a the same dictionary and NaN values (np.nan)
#otherwise does not fix it
#all_features[4589], 4689, 5360 are the ones

In [261]:
type(all_features[4589])

{'danceability': 0.747,
 'energy': 0.611,
 'key': 11,
 'loudness': -8.775,
 'mode': 0,
 'speechiness': 0.0475,
 'acousticness': 0.558,
 'instrumentalness': 0.0775,
 'liveness': 0.329,
 'valence': 0.348,
 'tempo': 120.034,
 'type': 'audio_features',
 'id': '5JDCV7Cj4P5qjDUy2A4mIc',
 'uri': 'spotify:track:5JDCV7Cj4P5qjDUy2A4mIc',
 'track_href': 'https://api.spotify.com/v1/tracks/5JDCV7Cj4P5qjDUy2A4mIc',
 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/5JDCV7Cj4P5qjDUy2A4mIc',
 'duration_ms': 166000,
 'time_signature': 4}

In [265]:
all_features[4589]= {'danceability': np.nan,'energy': np.nan,'key': np.nan,'loudness': np.nan,'mode': np.nan,
                     'speechiness': np.nan,'acousticness': np.nan,'instrumentalness': np.nan,'liveness': np.nan,
                     'valence': np.nan,'tempo': np.nan,'type': np.nan,'id': np.nan,'uri': np.nan,
                     'track_href': np.nan,'analysis_url': np.nan,'duration_ms': np.nan,'time_signature': np.nan}

In [269]:
all_features[4689]= {'danceability': np.nan,'energy': np.nan,'key': np.nan,'loudness': np.nan,'mode': np.nan,
                     'speechiness': np.nan,'acousticness': np.nan,'instrumentalness': np.nan,'liveness': np.nan,
                     'valence': np.nan,'tempo': np.nan,'type': np.nan,'id': np.nan,'uri': np.nan,
                     'track_href': np.nan,'analysis_url': np.nan,'duration_ms': np.nan,'time_signature': np.nan}

In [268]:
all_features[5360]= {'danceability': np.nan,'energy': np.nan,'key': np.nan,'loudness': np.nan,'mode': np.nan,
                     'speechiness': np.nan,'acousticness': np.nan,'instrumentalness': np.nan,'liveness': np.nan,
                     'valence': np.nan,'tempo': np.nan,'type': np.nan,'id': np.nan,'uri': np.nan,
                     'track_href': np.nan,'analysis_url': np.nan,'duration_ms': np.nan,'time_signature': np.nan}

In [270]:
features_spoty= pd.DataFrame(all_features)
features_spoty.head()

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,0.2210,0.1260,0.0,-25.427,1.0,0.0447,0.989,0.897,0.1020,0.2160,133.630,audio_features,4SFBV7SRNG2e2kyL1F6kjU,spotify:track:4SFBV7SRNG2e2kyL1F6kjU,https://api.spotify.com/v1/tracks/4SFBV7SRNG2e...,https://api.spotify.com/v1/audio-analysis/4SFB...,139307.0,4.0
1,0.0811,0.0122,4.0,-32.654,0.0,0.0511,0.902,0.308,0.0648,0.0384,74.554,audio_features,2kAgCRZPG3YQR2VMqRvLmb,spotify:track:2kAgCRZPG3YQR2VMqRvLmb,https://api.spotify.com/v1/tracks/2kAgCRZPG3YQ...,https://api.spotify.com/v1/audio-analysis/2kAg...,935360.0,4.0
2,0.0939,0.0336,2.0,-24.041,0.0,0.0606,0.927,0.830,0.0954,0.0516,67.359,audio_features,1upQiytDIEZfl9ItruoXuC,spotify:track:1upQiytDIEZfl9ItruoXuC,https://api.spotify.com/v1/tracks/1upQiytDIEZf...,https://api.spotify.com/v1/audio-analysis/1upQ...,207267.0,3.0
3,0.2300,0.0487,5.0,-27.675,1.0,0.0412,0.986,0.945,0.0679,0.1010,134.839,audio_features,0H2VhGUC3P3hvwz8rdGIpC,spotify:track:0H2VhGUC3P3hvwz8rdGIpC,https://api.spotify.com/v1/tracks/0H2VhGUC3P3h...,https://api.spotify.com/v1/audio-analysis/0H2V...,203656.0,5.0
4,0.2050,0.0525,1.0,-23.841,1.0,0.0327,0.980,0.906,0.1030,0.0511,88.316,audio_features,419qlOGNXoyICx4jO0hqaj,spotify:track:419qlOGNXoyICx4jO0hqaj,https://api.spotify.com/v1/tracks/419qlOGNXoyI...,https://api.spotify.com/v1/audio-analysis/419q...,174653.0,4.0


### 3.3. Put all together

In [272]:
tracks_spoty =tracks_spoty.reset_index(drop=True)

In [273]:
features_spoty = features_spoty.reset_index(drop=True)

In [274]:
display(tracks_spoty.tail()) 
display(features_spoty.tail()) 

,title,Id,artist
10666,Hermit the Frog,4Zcz6saEkOII3PlXd9gN3o,MARINA
10667,deja vu,61KpQadow081I2AsbeLcsb,Olivia Rodrigo
10668,WHOLE LOTTA MONEY,5yorXJWdBan1Vlh116ZtQ7,BIA
10669,Slumber Party (feat. Princess Nokia),11ZulcYY4lowvcQm4oe3VJ,Ashnikko
10670,telepatía,6tDDoYIxWvMLTdKpjFkc1B,Kali Uchis


,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
10666,0.609,0.679,0.0,-4.545,1.0,0.0312,0.24300,0.000000,0.199,0.487,122.034,audio_features,4Zcz6saEkOII3PlXd9gN3o,spotify:track:4Zcz6saEkOII3PlXd9gN3o,https://api.spotify.com/v1/tracks/4Zcz6saEkOII...,https://api.spotify.com/v1/audio-analysis/4Zcz...,215960.0,4.0
10667,0.439,0.610,9.0,-7.236,1.0,0.1160,0.59300,0.000011,0.341,0.172,181.088,audio_features,61KpQadow081I2AsbeLcsb,spotify:track:61KpQadow081I2AsbeLcsb,https://api.spotify.com/v1/tracks/61KpQadow081...,https://api.spotify.com/v1/audio-analysis/61Kp...,215508.0,4.0
10668,0.897,0.371,1.0,-5.019,1.0,0.3680,0.09040,0.000000,0.325,0.441,81.008,audio_features,5yorXJWdBan1Vlh116ZtQ7,spotify:track:5yorXJWdBan1Vlh116ZtQ7,https://api.spotify.com/v1/tracks/5yorXJWdBan1...,https://api.spotify.com/v1/audio-analysis/5yor...,156005.0,4.0
10669,0.964,0.398,11.0,-8.981,0.0,0.0795,0.00151,0.000039,0.101,0.563,105.012,audio_features,11ZulcYY4lowvcQm4oe3VJ,spotify:track:11ZulcYY4lowvcQm4oe3VJ,https://api.spotify.com/v1/tracks/11ZulcYY4low...,https://api.spotify.com/v1/audio-analysis/11Zu...,178405.0,4.0
10670,0.653,0.524,11.0,-9.016,0.0,0.0502,0.11200,0.000000,0.203,0.553,83.970,audio_features,6tDDoYIxWvMLTdKpjFkc1B,spotify:track:6tDDoYIxWvMLTdKpjFkc1B,https://api.spotify.com/v1/tracks/6tDDoYIxWvML...,https://api.spotify.com/v1/audio-analysis/6tDD...,160191.0,4.0


In [339]:
spoty_df = tracks_spoty.join(features_spoty)
spoty_df.tail()

,title,Id,artist,danceability,energy,key,loudness,mode,speechiness,acousticness,...,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
10666,Hermit the Frog,4Zcz6saEkOII3PlXd9gN3o,MARINA,0.609,0.679,0.0,-4.545,1.0,0.0312,0.24300,...,0.199,0.487,122.034,audio_features,4Zcz6saEkOII3PlXd9gN3o,spotify:track:4Zcz6saEkOII3PlXd9gN3o,https://api.spotify.com/v1/tracks/4Zcz6saEkOII...,https://api.spotify.com/v1/audio-analysis/4Zcz...,215960.0,4.0
10667,deja vu,61KpQadow081I2AsbeLcsb,Olivia Rodrigo,0.439,0.610,9.0,-7.236,1.0,0.1160,0.59300,...,0.341,0.172,181.088,audio_features,61KpQadow081I2AsbeLcsb,spotify:track:61KpQadow081I2AsbeLcsb,https://api.spotify.com/v1/tracks/61KpQadow081...,https://api.spotify.com/v1/audio-analysis/61Kp...,215508.0,4.0
10668,WHOLE LOTTA MONEY,5yorXJWdBan1Vlh116ZtQ7,BIA,0.897,0.371,1.0,-5.019,1.0,0.3680,0.09040,...,0.325,0.441,81.008,audio_features,5yorXJWdBan1Vlh116ZtQ7,spotify:track:5yorXJWdBan1Vlh116ZtQ7,https://api.spotify.com/v1/tracks/5yorXJWdBan1...,https://api.spotify.com/v1/audio-analysis/5yor...,156005.0,4.0
10669,Slumber Party (feat. Princess Nokia),11ZulcYY4lowvcQm4oe3VJ,Ashnikko,0.964,0.398,11.0,-8.981,0.0,0.0795,0.00151,...,0.101,0.563,105.012,audio_features,11ZulcYY4lowvcQm4oe3VJ,spotify:track:11ZulcYY4lowvcQm4oe3VJ,https://api.spotify.com/v1/tracks/11ZulcYY4low...,https://api.spotify.com/v1/audio-analysis/11Zu...,178405.0,4.0
10670,telepatía,6tDDoYIxWvMLTdKpjFkc1B,Kali Uchis,0.653,0.524,11.0,-9.016,0.0,0.0502,0.11200,...,0.203,0.553,83.970,audio_features,6tDDoYIxWvMLTdKpjFkc1B,spotify:track:6tDDoYIxWvMLTdKpjFkc1B,https://api.spotify.com/v1/tracks/6tDDoYIxWvML...,https://api.spotify.com/v1/audio-analysis/6tDD...,160191.0,4.0


In [340]:
spoty_df.columns

Index(['title', 'Id', 'artist', 'danceability', 'energy', 'key', 'loudness',
       'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness',
       'valence', 'tempo', 'type', 'id', 'uri', 'track_href', 'analysis_url',
       'duration_ms', 'time_signature'],
      dtype='object')

In [341]:
#Drop extra columns

spoty_df =spoty_df.drop(columns=["type","time_signature","analysis_url","track_href","Id"])

In [342]:
#Arrange the columns in the same orther as the Hotsong DB and wiht same colum names

spoty_df = spoty_df.reindex(columns=['title', 'artist', 'danceability', 'energy', 'key', 'loudness', 'mode',
       'speechiness', 'acousticness', 'instrumentalness', 'liveness',
       'valence', 'tempo', 'id', 'uri', 'duration_ms'])

In [343]:
#I am gonna delete the rows with the NaN values:
is_NaN = spoty_df.isnull()
row_has_NaN = is_NaN.any(axis=1)
rows_with_NaN = spoty_df[row_has_NaN]
rows_with_NaN 

,title,artist,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,id,uri,duration_ms
4589,Memories (feat. Kid Cudi) - 2021 Remix,David Guetta,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4689,Memories (feat. Kid Cudi) - 2021 Remix,David Guetta,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5360,One Jump Ahead (Reprise 2),Mena Massoud,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [344]:
#I will drop them:[4689,:],[5360,:]

spoty_df = spoty_df.drop([4589,4689,5360])

In [333]:
len(spoty_df)

10017

In [345]:
#I will also drop duplicates:

spoty_df = spoty_df.drop_duplicates()

In [346]:
len(spoty_df) # Final size 10017

10017

In [347]:
spoty_df.head()

,title,artist,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,id,uri,duration_ms
0,"The Well-Tempered Clavier: Book 1, BWV 846-869...",Johann Sebastian Bach,0.2210,0.1260,0.0,-25.427,1.0,0.0447,0.989,0.897,0.1020,0.2160,133.630,4SFBV7SRNG2e2kyL1F6kjU,spotify:track:4SFBV7SRNG2e2kyL1F6kjU,139307.0
1,The Lark Ascending,Ralph Vaughan Williams,0.0811,0.0122,4.0,-32.654,0.0,0.0511,0.902,0.308,0.0648,0.0384,74.554,2kAgCRZPG3YQR2VMqRvLmb,spotify:track:2kAgCRZPG3YQR2VMqRvLmb,935360.0
2,Handel / Orch. Hale: Keyboard Suite in D Minor...,George Frideric Handel,0.0939,0.0336,2.0,-24.041,0.0,0.0606,0.927,0.830,0.0954,0.0516,67.359,1upQiytDIEZfl9ItruoXuC,spotify:track:1upQiytDIEZfl9ItruoXuC,207267.0
3,"Bagatelle No. 25 in A Minor, ""Für Elise"", WoO 59",Ludwig van Beethoven,0.2300,0.0487,5.0,-27.675,1.0,0.0412,0.986,0.945,0.0679,0.1010,134.839,0H2VhGUC3P3hvwz8rdGIpC,spotify:track:0H2VhGUC3P3hvwz8rdGIpC,203656.0
4,"Rhapsody On A Theme Of Paganini, Op.43: Variat...",Sergei Rachmaninoff,0.2050,0.0525,1.0,-23.841,1.0,0.0327,0.980,0.906,0.1030,0.0511,88.316,419qlOGNXoyICx4jO0hqaj,spotify:track:419qlOGNXoyICx4jO0hqaj,174653.0


## 3. PUT BOTH DB TOGETHER FOR K-MEANS AND GET LABELS

### 3.1. Concatenate data frames

In [380]:
all_df = pd.concat([hot_songs_df, spoty_df], axis=0)

In [381]:
all_df.shape

(10117, 16)

In [382]:
all_df.head()

,title,artist,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,id,uri,duration_ms
0,Butter,BTS,0.759,0.459,8.0,-5.187,1.0,0.0948,0.00323,0.0,0.0906,0.695,109.997,2bgTY4UwhfBYhGT4HUYStN,spotify:track:2bgTY4UwhfBYhGT4HUYStN,164442.0
1,Good 4 U,Olivia Rodrigo,0.556,0.661,6.0,-5.052,0.0,0.2040,0.30000,0.0,0.1010,0.668,168.560,6PERP62TejQjgHu81OHxgM,spotify:track:6PERP62TejQjgHu81OHxgM,178148.0
2,Levitating,Dua Lipa Featuring DaBaby,0.702,0.825,6.0,-3.787,0.0,0.0601,0.00883,0.0,0.0674,0.915,102.977,463CkQjx2Zk1yXoBuierM9,spotify:track:463CkQjx2Zk1yXoBuierM9,203064.0
3,Peaches,Justin Bieber Featuring Daniel Caesar & Giveon,0.677,0.696,0.0,-6.181,1.0,0.1190,0.32100,0.0,0.4200,0.464,90.030,4iJyoBOLtHqaGxP12qzhQI,spotify:track:4iJyoBOLtHqaGxP12qzhQI,198082.0
4,Leave The Door Open,Silk Sonic (Bruno Mars & Anderson .Paak),0.586,0.616,5.0,-7.964,1.0,0.0324,0.18200,0.0,0.0927,0.719,148.088,7MAibcTli4IisCtbHKrGMh,spotify:track:7MAibcTli4IisCtbHKrGMh,242096.0


### 5.2 Select the audio features interested (train model in another file)

In [322]:
all_features_4_cluster =all_df[['danceability', 'energy', 'key', 'loudness', 'mode','speechiness', 'acousticness', 'instrumentalness', 'liveness','valence', 'tempo']]

In [323]:
all_features_4_cluster.tail()

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo
10665,0.346,0.944,9.0,-9.874,1.0,0.0485,0.000042,0.565000,0.0546,0.403,179.630
10666,0.609,0.679,0.0,-4.545,1.0,0.0312,0.243000,0.000000,0.1990,0.487,122.034
10668,0.897,0.371,1.0,-5.019,1.0,0.3680,0.090400,0.000000,0.3250,0.441,81.008
10669,0.964,0.398,11.0,-8.981,0.0,0.0795,0.001510,0.000039,0.1010,0.563,105.012
10670,0.653,0.524,11.0,-9.016,0.0,0.0502,0.112000,0.000000,0.2030,0.553,83.970


In [324]:
###export the file

all_features_4_cluster.to_csv("../data/audio_features_for_cluster.csv")
#continue working in other file

In [ ]:
### Now I must go to the File, Scaling and training

### 4.4 Add labels and export each DB

In [313]:
#retrieve scaled table of the audio features with an additional colums for groups
audio_features_for_cluster_labeled = pd.read_csv("../data/audio_features_for_cluster_labeled.csv",index_col=False)
audio_features_for_cluster_labeled.shape

(10117, 13)

In [327]:
audio_features_for_cluster_labeled.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10117 entries, 0 to 10116
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Unnamed: 0        10117 non-null  int64  
 1   danceability      10117 non-null  float64
 2   energy            10117 non-null  float64
 3   key               10117 non-null  float64
 4   loudness          10117 non-null  float64
 5   mode              10117 non-null  float64
 6   speechiness       10117 non-null  float64
 7   acousticness      10117 non-null  float64
 8   instrumentalness  10117 non-null  float64
 9   liveness          10117 non-null  float64
 10  valence           10117 non-null  float64
 11  tempo             10117 non-null  float64
 12  cluster           10117 non-null  int64  
dtypes: float64(11), int64(2)
memory usage: 1.0 MB


In [355]:
#a bit annoying it appears a new column all the time
#Store column caterogry in labels to add it to the
labels = audio_features_for_cluster_labeled.iloc[:,-1]

In [378]:
labels[10116]

6

In [379]:
labels.isnull().value_counts()

False    10117
Name: cluster, dtype: int64

In [383]:
#Add labels column to the db
all_df["category"] = labels

In [384]:
all_df.info() #its seems ok the 100 first, that were the HoT100

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10117 entries, 0 to 10670
Data columns (total 17 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   title             10117 non-null  object 
 1   artist            10117 non-null  object 
 2   danceability      10117 non-null  float64
 3   energy            10117 non-null  float64
 4   key               10117 non-null  float64
 5   loudness          10117 non-null  float64
 6   mode              10117 non-null  float64
 7   speechiness       10117 non-null  float64
 8   acousticness      10117 non-null  float64
 9   instrumentalness  10117 non-null  float64
 10  liveness          10117 non-null  float64
 11  valence           10117 non-null  float64
 12  tempo             10117 non-null  float64
 13  id                10117 non-null  object 
 14  uri               10117 non-null  object 
 15  duration_ms       10117 non-null  float64
 16  category          9595 non-null   float6

In [385]:
all_df['category'].values

array([14.,  5.,  6., ..., nan, nan, nan])

In [386]:
#Obtain our original HotSong DF with labels
hot_songs_df= all_df.iloc[:100,:]

In [387]:
#Obtain our original Spoty DF with labels
spoty_df = all_df.iloc[100:,:]

In [388]:
is_NaN = spoty_df.isnull()
row_has_NaN = is_NaN.any(axis=1)
rows_with_NaN = spoty_df[row_has_NaN]
rows_with_NaN 

,title,artist,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,id,uri,duration_ms,category
10117,PUNANI,6ix9ine,0.841,0.541,9.0,-7.121,1.0,0.2290,0.018600,0.000000,0.1230,0.420,97.961,41xKiK40e7MFu1bqcGSz1d,spotify:track:41xKiK40e7MFu1bqcGSz1d,115255.0,NaN
10118,Vastuu,Turisti,0.703,0.623,1.0,-6.906,1.0,0.2430,0.105000,0.000000,0.1210,0.813,179.114,6HacZ1Cd0NPtMWvkUFs0n8,spotify:track:6HacZ1Cd0NPtMWvkUFs0n8,195212.0,NaN
10119,Pretty Girls,iann dior,0.732,0.511,6.0,-6.779,1.0,0.0541,0.276000,0.000000,0.0916,0.620,96.946,3fKdTr6Uv8HTb92Ih3QWxa,spotify:track:3fKdTr6Uv8HTb92Ih3QWxa,133608.0,NaN
10120,Lying (feat. Lil Tjay),PRETTYMUCH,0.594,0.750,11.0,-3.693,0.0,0.0625,0.003670,0.000000,0.1660,0.328,142.023,6x8V3kUNlhVbwRbVP0QeSS,spotify:track:6x8V3kUNlhVbwRbVP0QeSS,203676.0,NaN
10121,Ghost Town,Kanye West,0.573,0.545,7.0,-4.247,0.0,0.0288,0.096900,0.000000,0.3290,0.297,83.378,7vgTNTaEz3CsBZ1N4YQalM,spotify:track:7vgTNTaEz3CsBZ1N4YQalM,271387.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10665,Freaks,Surf Curse,0.346,0.944,9.0,-9.874,1.0,0.0485,0.000042,0.565000,0.0546,0.403,179.630,0HUTL8i4y4MiGCPId7M7wb,spotify:track:0HUTL8i4y4MiGCPId7M7wb,147036.0,NaN
10666,Hermit the Frog,MARINA,0.609,0.679,0.0,-4.545,1.0,0.0312,0.243000,0.000000,0.1990,0.487,122.034,4Zcz6saEkOII3PlXd9gN3o,spotify:track:4Zcz6saEkOII3PlXd9gN3o,215960.0,NaN
10668,WHOLE LOTTA MONEY,BIA,0.897,0.371,1.0,-5.019,1.0,0.3680,0.090400,0.000000,0.3250,0.441,81.008,5yorXJWdBan1Vlh116ZtQ7,spotify:track:5yorXJWdBan1Vlh116ZtQ7,156005.0,NaN
10669,Slumber Party (feat. Princess Nokia),Ashnikko,0.964,0.398,11.0,-8.981,0.0,0.0795,0.001510,0.000039,0.1010,0.563,105.012,11ZulcYY4lowvcQm4oe3VJ,spotify:track:11ZulcYY4lowvcQm4oe3VJ,178405.0,NaN


In [390]:
hot_songs_df.to_csv("../data/hotsongs_db.csv")

In [391]:
spoty_df.to_csv("../data/spotify_db.csv")